In [ ]:
import os

import numpy as np
import pandas as pd

from utils import sph2cart, sph_normals, IPD

In [ ]:
total_radii = {3: [3.6258, 9.9, 16.1742],
               5: [2.5599, 5.5384, 9.9, 14.2616, 17.2401],
               7: [2.2122, 3.8936, 6.6127, 9.9, 13.1873, 15.9064, 17.5878],
               9: [2.0579, 3.1281, 4.9317, 7.2735, 9.9, 12.5265, 14.8683, 16.6719, 17.7421]} 
frequencies = [2.4, 3.0, 3.5, 5.0, 6.0, 10.0]  # frequency
distances = [2.5, 5, 10, 50, 150]  # separation distance
deg = 33  # degree of integration
edge_length = 0.02
rows = 0
for key in total_radii.keys():
    rows += len(total_radii[key])
rows *= len(frequencies) * len(distances)
col_names = ['radius [cm]', 'frequency [GHz]', 'distance [mm]', 'col_points', 'sPDn_pln [W/m2]', 'sPDn_sph [W/m2]']
cols = len(col_names)

In [ ]:
curvature = np.empty((rows, cols))
row_idx = 0
for col_point in total_radii.keys():
    radii = total_radii[col_point]
    for radius in radii:
        for frequency in frequencies:
            for distance in distances:
                sPDn_pln, sPDn_sph = IPD(radius/100, frequency*1e9, -distance/1000, deg, edge_length)
                curvature[row_idx] = radius, frequency, distance, col_point, sPDn_pln, sPDn_sph
                message = (f'[{row_idx+1}/{rows} {round((row_idx+1)/rows*100, 2)}%]\t'
                           f'sPDn_pln = {round(sPDn_pln, 2)} W/m2\t'
                           f'sPDn_sph = {round(sPDn_sph, 2)} W/m2')
                print(message, end='\r')
                row_idx += 1
                
fname = 'curvature'
df = pd.DataFrame(data=eval(fname), columns=col_names)
df.to_csv(os.path.join('data', fname+'.csv'), index_label='index')